In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import os, sys
sys.path.append('../')
import seaborn as sns
import warnings
from utils.utils import set_seed
import pickle
warnings.filterwarnings("ignore")
set_seed(42)

In [ ]:
def cal_mag_susceptibility(M, L, T=2.27):
    M = np.abs(M)  
    mag_susceptibility = np.mean(M ** 2) - np.mean(M) ** 2
    mag_susceptibility = mag_susceptibility * L ** 2 / T 
    return mag_susceptibility

In [ ]:
device = torch.device('cuda:7')

In [ ]:
M_equilibrium = {}
predict_tra = {}
M_predict = {}


L_list = [16, 32, 64, 128]
for L in L_list:
    predict_tra[f'L{L}'] = []
    M_predict[f'L{L}'] = []

In [ ]:
length = 32000
folder = '../checkpoints_T_2.27/'
paths = os.listdir(folder)
with torch.no_grad():
    for L in L_list:
        
        ckpt_paths = [os.path.join('../checkpoints_T_2.27/', path) for path in paths if f'L_{L}' in path and 'SDE' in path]
        ckpt_paths = [os.path.join(path, 'model.pt') for path in ckpt_paths]
        
        for ckpt_path in ckpt_paths: 
            model = torch.load(ckpt_path, map_location=device)
            z0_train = torch.load(f'../data/ising_patch_L_16_L_{L}_T_2.27/z0_train.pt', map_location=device)
            if L == 16:
                train_dt = torch.load(f'../raw_data/L{L}_MC32000_h0.0_T2.27/time_step_train.pt', map_location=device) # [n_tra, length_per_tra]
            else:
                train_dt = torch.load(f'../raw_data_upsample/scaleup_L{L}_h0.0_T2.27/time_step_train_partial.pt', map_location=device) # [n_tra, length_per_tra]

            val_dt = torch.load(f'../raw_data/L{L}_MC32000_h0.0_T2.27/time_step_val.pt', map_location=device)
            mean_train_dt = torch.mean(train_dt)
            val_dt = val_dt / mean_train_dt
            mean_dt = torch.mean(val_dt)
            
            initial = z0_train[:, 0]
            predict = model.predict(initial, length, dt=mean_dt)

            nan_mask = torch.isnan(predict).any(dim=(1, 2))  # Check for NaN across time and feature dimensions
            if nan_mask.any():
                print(f'Found {nan_mask.sum().item()} trajectories with NaN values, removing them...')
                predict = predict[~nan_mask]  # Keep only trajectories without NaN
                print(f'Remaining trajectories: {predict.shape[0]}')
            
            predict_tra[f'L{L}'].append(predict)


            M_equilibrium[f'L{L}'] = torch.load(f'../raw_data/L{L}_MC32000_h0.0_T2.27/M_equilibrium.pt').detach().cpu().numpy()
            M = predict[:, (length // 2):, 0].detach().cpu().numpy()
            M = np.abs(M)
            M = np.clip(M, 0, 1)
            M_predict[f'L{L}'].append(M)

In [ ]:
# dump the results
with open('plot_data/M_equilibrium_T_2.27.pkl', 'wb') as f:
    pickle.dump(M_equilibrium, f)
with open('plot_data/M_predict_T_2.27.pkl', 'wb') as f:
    pickle.dump(M_predict, f)